## 1. Setting up Kaggle for the DL Model and downloading the dataset

In [ ]:
! pip install kaggle

In [ ]:
# Configuring the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# Downloading Dataset using API
!kaggle competitions download -c cifar-10

In [ ]:
! ls

In [ ]:
# Extracting the compressed folder
from zipfile import ZipFile
dataset = "/content/cifar-10.zip"

with ZipFile(dataset, 'r') as zip:
  zip.extractall()
  print('The dataset is successfully extracted')


In [ ]:
!ls


In [ ]:
# Installing py7zr library to extract train.7z file
! pip install py7zr

In [ ]:
import py7zr
archive = py7zr.SevenZipFile('/content/train.7z', mode='r')
archive.extractall()
archive.close()

In [ ]:
! ls

## 2. Importing the Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split

In [ ]:
# Having a look at training data
filenames = os.listdir("/content/train")
print(filenames)

In [ ]:
type(filenames)

In [ ]:
len(filenames)

In [ ]:
print(filenames[0:5])
print(filenames[-5:])

## 3. Processing the Labels

In [ ]:
labels_df = pd.read_csv("/content/trainLabels.csv")
labels_df.head(10)

In [ ]:
labels_df.shape

In [ ]:
labels_df[labels_df['id'] == 2351] #example

In [ ]:
labels_df['label'].value_counts()

In [ ]:
# Encoding label strings to numerics using dictionaries' key value pairs
labels_dictionary ={'airplane': 0, 'automobile': 1, 'bird': 2, 'cat': 3, 'deer': 4, 'dog': 5, 'frog': 6, 'horse': 7, 'ship': 8, 'truck': 9 }

labels = [labels_dictionary[i] for i in labels_df['label']]


In [ ]:
type(labels)

In [ ]:
print(labels[0:5]) #example

In [ ]:
len(labels)

## 4. Processing the images training dataset

In [ ]:
# Displaying sample image
import cv2
from google.colab.patches import cv2_imshow # This step is required because colab does not support cv2.mshow function since it creates another pop up window

img = cv2.imread('/content/train/1.png')
cv2_imshow(img)

In [ ]:
labels_df.head()

In [ ]:
id_list = list(labels_df['id']) # creating an iteratvive array

In [ ]:
print(id_list)

In [ ]:
# Image Processing
# Converting 50,000 Images to 50,000 numpy arrays and storing them in a list
# All the images must be of the same size, if not we have to use reshape function to reshapeevery image to a common dimension value
# Here all the images are already in same size 32*32
train_data_folder = '/content/train/'
data = []

for id in id_list:
  image = Image.open(train_data_folder + str(id) + '.png') # opening images using PIL Library
  image = np.array(image) # converting images to numpy arrays
  data.append(image) # Storing image numpy arrays in a list



In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
type(data[0])

In [ ]:
data[0].shape

In [ ]:
print(data[0]) # Example

In [ ]:
# Converting the labels list and images list to numpy arrays
X = np.array(data)
Y = np.array(labels)

In [ ]:
type(X)

In [ ]:
print(X.shape)
print(Y.shape)

## 5. Splitting Training data into train and test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 2)

In [ ]:
print(X.shape)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

## 6. Scaling the Images data between 0 to 1

In [ ]:
X_train_scaled = X_train/255
X_test_scaled = X_test/255

In [ ]:
print(X_train_scaled)

## 7. Building the Neural Network

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
num_of_classes = 10
# Setting up the layers of Neural Network

model = keras.Sequential([
    keras.layers.Flatten(input_shape = (32,32,3)),
    keras.layers.Dense(64, activation = 'relu'), # hidden layer
    keras.layers.Dense(num_of_classes, activation = 'softmax') # for multiclass classification we use here softmax as a activation function
])

In [ ]:
# Compile the neural network
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

In [ ]:
# Training the neural network
model.fit(X_train_scaled, Y_train, validation_split= 0.1, epochs = 20)

## 8. Training the dataset using ResNet50 Model

In [ ]:
# Importing all the libraries and the functions required to build ResNet50
from tensorflow.keras import Sequential, layers, models
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras import optimizers


In [ ]:
convolutional_base = ResNet50(weights = 'imagenet', include_top = False, input_shape = (256,256,3)) # include_top = False suggests to not include the output layer of the network
convolutional_base.summary()

In [ ]:
model = models.Sequential()
model.add(layers.UpSampling2D((2,2))) #scale the image by 2x to get a image with dimension 64*64
model.add(layers.UpSampling2D((2,2))) #images output with dimension 128*128
model.add(layers.UpSampling2D((2,2))) #images output with dimension 256*256 which is required for ResNet50 model
model.add(convolutional_base) #using ResNet50 model
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation = 'relu'))
model.add(layers.Dropout(0.5)) # To reduce the risk of overfitting
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(num_of_classes, activation = 'softmax'))



In [ ]:
model.compile(optimizer = optimizers.RMSprop(learning_rate=2e-5), loss = 'sparse_categorical_crossentropy', metrics = ['acc'])

In [ ]:
history = model.fit(X_train_scaled, Y_train, validation_split=0.1, epochs = 10)

In [ ]:
# Plotting our loss and accuracy change
h = history
# loss value
plt.plot(h.history['loss'], label = 'train loss')
plt.plot(h.history['val_loss'], label = 'validation loss')
plt.legend()
plt.show()

# Accuracy value
plt.plot(h.history['acc'], label = 'train accuracy')
plt.plot(h.history['val_acc'], label = 'validation accuracy')
plt.legend()
plt.show()

